# March & Sanchis Investments

## Try me
[![Open In Colab](../../_static/colabs_badge.png)](https://colab.research.google.com/github/ffraile/operations-research-notebooks/blob/main/docs/source/CLP/solved/investments%20(Solved%20GRB).ipynb)[![Binder](../../_static/binder_badge.png)](https://mybinder.org/v2/gh/ffraile/operations-research-notebooks/main?labpath=docs%2Fsource%2FCLP%2Fsolved%2Finvestments%20(Solved%20GRB).ipynb)

## Problem Definition
Suppose you are a financial advisor at March & Sanchis Investments Ltd. and your client has €1,000,000 to invest in four different types of assets: stocks, bonds, real estate, and crypto assets, with the Expected Returns and risk (an estimation of the risk per euro invested) expressed in the table below:

| Asset Type	 | Expected Return (%)	| Risk (%) |
|--------------|----------------------|----------|
| Stocks	     |10	                  | 1.25     |
| Bonds        |6	                    | 0.5      |
| Real Estate  |8	                    | 0.7      |
| Crypto Assets|15	                  | 2.1      |

Your task is to help the client select the optimal portfolio of investments, maximizing the Expected Return. Assume that the investments are independent and take into account the following constraints:

- **Budget:** The total investment cannot exceed the budget of the client (€1,000,000)
- **Personal Preferences:** The client wants to invest at least €200,000 in real estate due to personal interests in owning property, and does not want to invest more than €400,000 in crypto assets as she would like to be cautious and limit exposure.
- **Risk:** The overall risk estimation cannot exceed 12% of the expected return

To model this as a continuous linear programming problem, let:

- $x_1$ = Amount invested in stocks (€)
- $x_2$ = Amount invested in bonds (€)
- $x_3$ = Amount invested in real estate (€)
- $x_4$ = Amount invested in crypto assets (€)

Then, we want to maximize the expected return, which is given by:

$\max z = 0.1*x_1 + 0.06*x_2 + 0.08*x_3 + 0.15*x_4$ (Expected return in €)

Subject to::


$x_1 + x_2 + x_3 + x_4 = 1,000,000$ (total investment must equal $1,000,000)

$0.0125*x_1 + 0.005*x_2 + 0.007*x_3 + 0.021*x_4 ≤ 0.12*(0.1*x_1 + 0.06*x_2 + 0.08*x_3 + 0.15*x_4)$ (risk limit: the risk of the portfolio, cannot exceed 12% of the total expected return)

$x_3 ≥ 200,000$ (minimum investment in real estate: the client wants to invest at least €200,000 in real estate)

$x_4 ≤ 400,000$ (maximum investment in crypto assets: the client is cautious about the high risk of crypto assets and does not want to invest more than $400,000 in them)

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import pulp

model = pulp.LpProblem("Maximising_investments", pulp.LpMaximize)

x = pulp.LpVariable.dicts("x",
                            (i+1 for i in range(4)),
                            lowBound=0,
                            upBound=1000000,
                            cat='Continuous')



model += 0.1*x[1] + 0.06*x[2] + 0.08*x[3] + 0.15*x[4]

# Constraints
## The total investment must equal $1,000,000
model += x[1] + x[2] + x[3] + x[4] == 1000000, "Maximum_budget"

## The estimated risk, cannot exceed 12% of the total expected return
model += 0.0125*x[1] + 0.005*x[2] + 0.007*x[3] + 0.021*x[4] <= 0.12*(0.1*x[1] + 0.06*x[2] + 0.08*x[3] + 0.15*x[4]), "Risk"

## The client wants to invest at least $200,000 in real estate
model += x[3] >= 200000, "Minimum_investment_in_real_estate"

## The client is cautious about the high risk of crypto assets and does not want to invest more than $400,000 in them
model += x[4] <= 400000, "Maximum_investment_in-crypto assets"

model.solve(solver=pulp.GUROBI(msg = 0))
pulp.LpStatus[model.status]

expected_return = pulp.value(model.objective)
display(Markdown("Expected resturn is %0.2f €"%expected_return))

display(Markdown("The following table shows the decision variables: "))
var_df = pd.DataFrame.from_dict(x, orient="index",
                                columns = ["Variables"])
var_df["Solution (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.X))
var_df["Reduced cost (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.RC))
var_df["Objective Coefficient (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.Obj))
var_df["Objective Lower bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjLow) if item.solverVar.SAObjLow > -0.1 else "-Inf" )
var_df["Objective Upper bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjUp) if item.solverVar.SAObjUp != item.solverVar.UB else "Inf")


display(var_df)


const_dict = dict(model.constraints)
con_df = pd.DataFrame.from_records(list(const_dict.items()), exclude=["Expression"], columns=["Constraint", "Expression"])
con_df["Right Hand Side"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.RHS))
con_df["Slack"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Slack))
con_df["Shadow Price"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Pi))
con_df["Min RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSLow) if const_dict[item].solverConstraint.SARHSLow > -1e10 else "-Inf")
con_df["Max RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSUp) if const_dict[item].solverConstraint.SARHSUp < 1e10 else "Inf" )


display(Markdown("The following table shows the constraints: "))
display(con_df)



Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-23


Expected resturn is 110322.58 €

The following table shows the decision variables: 

,Variables,Solution (GRB),Reduced cost (GRB),Objective Coefficient (GRB),Objective Lower bound (GRB),Objective Upper bound (GRB)
1,x_1,116129.03,0.00,0.10,0.08,0.12
2,x_2,0.00,-0.02,0.06,-Inf,0.08
3,x_3,483870.97,0.00,0.08,0.05,0.10
4,x_4,400000.00,0.00,0.15,0.12,inf


The following table shows the constraints: 

,Constraint,Right Hand Side,Slack,Shadow Price,Min RHS,Max RHS
0,Maximum_budget,1000000.00,0.00,0.10,861538.46,2053846.15
1,Risk,0.00,0.00,6.45,-360.00,880.00
2,Minimum_investment_in_real_estate,200000.00,-283870.97,0.00,-Inf,483870.97
3,Maximum_investment_in_crypto_assets,400000.00,0.00,0.03,48000.00,464285.71
